In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
from tqdm import tqdm
import numpy as np

DATA_DIR = './MNIST_data'


In [2]:




def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                                                strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)





In [ ]:
# Import data
print("here")
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)
print("there")
# Create the model
x = tf.placeholder(tf.float32, [None, 784]) # 784=28*28

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net

# Reshape to use within a convolutional neural net.
# Last dimension is for "features" - there is only one here, since images are
# grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
x_image = tf.reshape(x, [-1, 28, 28, 1])

# First convolutional layer - maps one grayscale image to 32 feature maps.
W_conv1 = weight_variable([5, 5, 1, 32])
# each kernel has spatial resolution od (5x5), inputs tensor has 1 channel
# train 32 convolutional kernels
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# Pooling layer - downsamples by 2X.
h_pool1 = max_pool_2x2(h_conv1)

# Second convolutional layer -- maps 32 feature maps to 64.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# Second pooling layer.
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Map the 1024 features to 10 classes, one for each digit
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


# Neccessities for training process
cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# Only for results monitoring
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_acc_all = []
    test_acc_all = []
    for i in tqdm(range(20000)):
        batch = mnist.train.next_batch(32)

        if i % 100 == 0:
            train_acc_ = accuracy.eval(
                feed_dict={
                    x: mnist.train.images[:5000], y_: mnist.train.labels[:5000], keep_prob: 1.0})
            test_acc_ = accuracy.eval(
                feed_dict={
                    x: mnist.test.images[:5000], y_: mnist.test.labels[:5000], keep_prob: 1.0})
            train_acc_all.append(train_acc_)
            test_acc_all.append(test_acc_)
            print('step %d, train acc=%.4f, test acc=%.4f' % (i, train_acc_, test_acc_))

            # Save tuple of arrays to a file
            np.savez('train_test_results', train=np.array(train_acc_all), test=np.array(test_acc_all))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})


here
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
there


  0%|          | 1/20000 [00:02<16:15:07,  2.93s/it]

step 0, train acc=0.0940, test acc=0.1050


  1%|          | 104/20000 [00:07<46:23,  7.15it/s] 

step 100, train acc=0.8510, test acc=0.8252


...

  6%|▌         | 1105/20000 [00:58<36:54,  8.53it/s]

step 1100, train acc=0.9602, test acc=0.9494


  6%|▌         | 1205/20000 [01:03<35:17,  8.88it/s]

step 1200, train acc=0.9648, test acc=0.9488


  6%|▋         | 1299/20000 [01:06<10:03, 30.99it/s]